In [ ]:
%pip install selenium bs4 tqdm sqlalchemy tabula-py psycopg2-binary lxml python-dotenv

In [ ]:
import os
import requests
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import Select
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from urllib.parse import urlparse
from bs4 import BeautifulSoup 
from datetime import datetime
import uuid
import re
from collections import defaultdict  # Add this line
from tqdm import tqdm

# Create the "tables" folder if it doesn't exist
tables_folder_path = "./tables"
if not os.path.exists(tables_folder_path):
    os.makedirs(tables_folder_path)


# Create the "matrizes" folder if it doesn't exist
pdfs_folder_path = "./courses_pdfs"
if not os.path.exists(pdfs_folder_path):
    os.makedirs(pdfs_folder_path)


In [ ]:
from sqlalchemy import create_engine, inspect
from sqlalchemy.sql import text
from dotenv import load_dotenv
import os

def wipe_database_tables():
    load_dotenv()

    # Create a connection to the PostgreSQL database
    conn_str = os.getenv('DB_URL')
    engine = create_engine(conn_str)

    # Create inspector
    inspector = inspect(engine)

    # Get table names, excluding unwanted tables
    unwanted = {'session', 'account', 'user', 'verification_token'}
    table_names = [e for e in inspector.get_table_names() if e not in unwanted]

    # Confirm with the user
    print(f"This will delete all data from the following tables:")
    print(", ".join(table_names))
    confirmation = input("Are you sure you want to proceed? (yes/no): ")

    if confirmation.lower() != 'yes':
        print("Operation cancelled.")
        return

    # Truncate tables
    with engine.connect() as connection:
        # Start a transaction
        with connection.begin():
            for table in table_names:
                # Use TRUNCATE with CASCADE to handle foreign key constraints
                connection.execute(text(f"TRUNCATE TABLE {table} CASCADE;"))
        
        print("All specified tables have been wiped.")

In [ ]:
from sqlalchemy import create_engine, inspect
from dotenv import load_dotenv
import os

load_dotenv()

# Create a connection to the PostgreSQL database
conn_str = os.getenv('DB_URL')
print(conn_str)
engine = create_engine(conn_str)

# create inspector
inspector = inspect(engine)

# get table names
unwanted = {'session', 'account', 'user', 'verification_token'}
table_names = [e for e in inspector.get_table_names() if e not in unwanted]
print(table_names)

# initialize an empty dictionary to hold the data
existing_data = {}

# iterate over all table names
for table in table_names:
    # read the data from the table and save it to the dictionary
    existing_data[table] = pd.read_sql(f'SELECT * FROM {table}', engine)


In [ ]:
import re
import unicodedata

def store_df(df, table):
    # Store the DataFrame in database
    df.to_sql(table, engine, index=False, if_exists='append')
    
    # Append the DataFrame to its CSV file
    df.to_csv(
        f"{tables_folder_path}/{table}.csv",
        mode="a",
        header=not os.path.exists(f"{tables_folder_path}/{table}.csv"),
        index=False,
    )


def format_course_name(text):
    # Remove accent marks
    text = ''.join(c for c in unicodedata.normalize('NFD', text) if unicodedata.category(c) != 'Mn')
    
    # Replace symbols with a hyphen
    text = re.sub(r'[^a-zA-Z0-9]+', '-', text)
    
    # Remove leading and trailing hyphens
    text = text.strip('-')
    
    # Convert to lowercase
    text = text.lower()
    
    return text


def get_discipline_prefix(code):
    match = re.match(r"([A-Z]{3})\d{3}", code)
    return match.group(1) if match else None

def handle_equivalencies(disciplines_data):
    equivalency_groups = defaultdict(list)
    
    for discipline in disciplines_data:
        key = (discipline['name'], get_discipline_prefix(discipline['code']))
        equivalency_groups[key].append(discipline)
    
    return [group for group in equivalency_groups.values() if len(group) > 1]

### Busca cursos e salva os .pdfs na pasta /matrizes

In [ ]:
# from tqdm import tqdm

# # Set up Chrome options
# chrome_options = Options()
# chrome_options.add_argument("--headless")  # Run Chrome in headless mode
# chrome_options.add_argument("--disable-gpu")
# chrome_options.add_argument("--no-sandbox")

# # Set up Chrome driver service
# chromedriver_path = (
#     "./chromedriver"  # Replace with the path to your chromedriver executable
# )
# service = Service()

# # Set up Chrome driver
# driver = webdriver.Chrome(service=service, options=chrome_options)

# # Navigate to the URL
# url = "https://www.escolha.ufop.br/cursos"
# driver.get(url)


# # Find elements with class "ufop-glossary-row"
# elements = driver.find_elements(By.CLASS_NAME, "ufop-glossary-row")

# # Extract the href links from child anchor 'a' tags
# links = []
# courses_dict = {"id": [], "code": [], "name": []}

# print("Buscando a lista de cursos da UFOP...")
# with tqdm(total=len(elements), desc="Progresso", ascii=True) as pbar:
#     for element in elements:
#         link_element = element.find_element(By.TAG_NAME, "a")
#         href = link_element.get_attribute("href")
#         links.append(href)

#         courses_dict["id"].append(str(uuid.uuid4()))
#         courses_dict["code"].append(format_course_name(link_element.text))
#         courses_dict["name"].append(link_element.text)
#         pbar.set_postfix(curso=f"{link_element.text}")
#         pbar.update(1)

# indexes_to_delete = []
# course_pdfs = []

# # Navigate to each link and download PDF files
# print("\nBuscando os links .pdf ...")
# with tqdm(total=len(links), desc="Progresso", ascii=True) as pbar:
#     for i, link in enumerate(links):
#         driver.get(link)

#         matriz_elements = driver.find_elements(
#             By.CLASS_NAME, "field-name-field-matriz-curricular"
#         )

#         for element in matriz_elements:
#             link_elements = element.find_elements(By.TAG_NAME, "a")

#             if len(link_elements) == 1:
#                 href = link_elements[0].get_attribute("href")
#                 course_pdfs.append({"course": courses_dict["code"][i], "link": href})
#                 continue

#             for link_element in link_elements:
#                 href = link_element.get_attribute("href")
#                 course_type = link_element.text
#                 course_name = f"{courses_dict['name'][i]} ({course_type})"
#                 course_code = format_course_name(course_name)
#                 courses_dict["id"].append(str(uuid.uuid4()))
#                 courses_dict["name"].append(course_name)
#                 courses_dict["code"].append(course_code)

#                 course_pdfs.append({"course": course_code, "link": href})
#             indexes_to_delete.append(i)

#         pbar.set_postfix(curso=f"{courses_dict['name'][i]}")
#         pbar.update(1)

# courses_dict["id"] = [
#     item for i, item in enumerate(courses_dict["id"]) if i not in indexes_to_delete
# ]
# courses_dict["code"] = [
#     item for i, item in enumerate(courses_dict["code"]) if i not in indexes_to_delete
# ]
# courses_dict["name"] = [
#     item for i, item in enumerate(courses_dict["name"]) if i not in indexes_to_delete
# ]

# valid_pdf_substrings = [".pdf", "codCurso="]
# for pdf in course_pdfs:
#     import os

# print("\nSalvando os arquivos .pdf dos cursos")
# with tqdm(total=len(course_pdfs), desc="Progresso", ascii=True) as pbar:

#     for pdf in course_pdfs:
#         if any(text in pdf["link"] for text in valid_pdf_substrings):
#             response = requests.get(pdf["link"])
#             parsed_url = urlparse(pdf["link"])
#             filename = f"{pdf['course']}.pdf"
#             file_path = os.path.join(pdfs_folder_path, filename)

#             # Check if the file already exists in the folder
#             if not os.path.exists(file_path):
#                 with open(file_path, "wb") as file:
#                     file.write(response.content)
        
#         pbar.set_postfix(salvando=f"{pdf['course']}.pdf")
#         pbar.update(1)
        
# # Quit the driver
# driver.quit()


### Salva os cursos encontrados no banco de dados

In [ ]:
# course_df = pd.DataFrame(courses_dict).sort_values(by="code", ascending=True)
# course_df["created_at"] = datetime.now()

# existing_course_codes = []
# # Check if 'course' key exists in existing_data
# if "course" in existing_data:
#     existing_course_codes = set(existing_data["course"]["code"])

# course_df = course_df[~course_df["code"].isin(existing_course_codes)]

# store_df(course_df, "course")
# print("Cursos salvos com sucesso!")

In [ ]:
wipe_database_tables()

In [ ]:
courses_dict = {'id': ['3047ed31-963f-4f72-977f-bc07126e010f', '3e25e994-04fe-47b5-bb93-b13746b2ee1d', '67355b65-cadf-4e5b-b178-2f325ffe8e6a', 'bc37098a-f50e-44cf-99f6-20349abd7360', '9705e1d4-2966-4b17-95cf-cda120129997', '6709b7f1-2ae8-4e96-aea8-816bcb4411c2', '8fe45895-3761-4bb3-9b17-b806637d638f', '42f17c5c-8dfe-4b69-8007-c5b741e495cb', '221c757d-2b6d-4e3c-8afb-9a1a65788f7d', 'e696737c-229f-4c56-b1ce-68fc0e83ee95', '275cdc83-793a-4b43-8ea4-71d36f3889cc', 'e837b4d1-550b-4b1f-ad2a-efdcb8d427dc', '63592dfa-b49c-4c91-8e72-bd0895c2bbcb', 'db4fa158-cf65-40e2-9b6f-9b181f1aadd7', 'b8ae4a8b-42e5-4d30-9b63-cb8001e59d49', '781c1f2c-99be-4103-81d2-743e6aa49782', 'fbb518a2-bce7-472d-b4b4-6bb32fd81853', '7373fd64-cceb-4e89-8ce4-b7f7fd05b0ed', '97f66c3d-7c1f-470c-9d4b-020d60ffe6aa', '104b5e46-fb02-484c-b208-06a6162c68ad', 'b9a4ea67-9ced-45ca-8de7-8a719b7a3d85', '7dfca9d3-b38d-46a2-86f4-c0cd2018694f', 'a174d77d-dcac-4c7d-a866-3fee3bbf55bb', 'b3899630-d91e-4129-834b-37a1c403d356', '3c70bd25-ff2d-4c31-a187-dda1be9d858a', '183a67c6-e4c5-4440-b258-fa55e63d2df1', '827ff4e4-2a2e-4273-89fa-f4c7e9c5f4dc', '894acc71-5cb8-477a-bbdb-8567f0215c21', '346d1ed8-cc3b-4bf8-8a96-f380c0dd2452', '841543ab-bc3f-489e-91c2-ab9b95e545c3', 'da525bd7-00a5-4c72-b37d-72d87fc7ec6c', '4b6c5510-c0f1-4d39-9379-b4a2c6b47c7d', 'eb3c9444-02a7-4a51-a4f7-67198b2521fa', 'd67c99ef-2656-44eb-b449-0709aa72c43b', '6b7923ec-5850-49d9-a132-c5b257f5fbe4', '1facc180-890f-49a5-b37f-50768b4f07f8', '7de1e05e-244f-49b6-9426-fe438bf09b62', 'ebbc5fb7-9596-4548-86e1-5fac09ebb599', '80904be4-f4fc-4364-9380-849b909ef380', '7d8634bf-1a36-425e-a453-b1b64c599d8e', 'c750d6da-acbb-4d1b-9951-e05dec51782a', 'd73a1748-222d-4d78-93db-25314d6b6db8', 'a56d95e3-ab68-48e7-8425-75b090305142', '61261050-8847-48fb-80e8-e6b726d73655', 'db4a1cb4-22aa-4728-ba87-87d74adb1753', '6792ec98-08b1-419e-8961-a45cdd8fb36e', '4085b8de-35a6-4abb-98f9-2ed9d6603f85', '7e9a6645-ecee-49d5-8e00-1f21863cb019', '2aa595b4-4176-493f-b720-7067206ef986', '0c483044-d409-4022-9b15-5224ba689d6b', '8077d3dd-47e4-4e00-a362-f807e10bda1f', 'bb0598d4-b967-4cca-a558-541191f048b0'], 'code': ['administracao', 'administracao-publica', 'administracao-publica-ead', 'arquitetura-e-urbanismo', 'ciencia-da-computacao', 'ciencia-e-tecnologia-de-alimentos', 'ciencias-economicas', 'direito', 'engenharia-ambiental', 'engenharia-civil', 'engenharia-de-computacao', 'engenharia-de-controle-e-automacao', 'engenharia-de-minas', 'engenharia-de-producao-jm', 'engenharia-de-producao-op', 'engenharia-eletrica', 'engenharia-geologica', 'engenharia-mecanica', 'engenharia-metalurgica', 'engenharia-urbana', 'estatistica', 'farmacia', 'geografia-ead', 'jornalismo', 'matematica-ead', 'medicina', 'museologia', 'musica', 'nutricao', 'pedagogia', 'pedagogia-ead', 'quimica', 'quimica-industrial', 'servico-social', 'sistemas-de-informacao', 'turismo', 'artes-cenicas-bacharelado', 'artes-cenicas-licenciatura', 'ciencias-biologicas-bacharelado', 'ciencias-biologicas-licenciatura', 'educacao-fisica-bacharelado', 'educacao-fisica-licenciatura', 'filosofia-bacharelado', 'filosofia-licenciatura', 'fisica-bacharelado', 'fisica-licenciatura', 'historia-bacharelado', 'historia-licenciatura', 'letras-bacharelado', 'letras-licenciatura', 'matematica-bacharelado', 'matematica-licenciatura'], 'name': ['Administração', 'Administração Pública', 'Administração Pública (EaD)', 'Arquitetura e Urbanismo', 'Ciência da Computação', 'Ciência e Tecnologia de Alimentos', 'Ciências Econômicas', 'Direito', 'Engenharia Ambiental', 'Engenharia Civil', 'Engenharia de Computação', 'Engenharia de Controle e Automação', 'Engenharia de Minas', 'Engenharia de Produção (JM)', 'Engenharia de Produção (OP)', 'Engenharia Elétrica', 'Engenharia Geológica', 'Engenharia Mecânica', 'Engenharia Metalúrgica', 'Engenharia Urbana', 'Estatística', 'Farmácia', 'Geografia (EaD)', 'Jornalismo', 'Matemática (EaD)', 'Medicina', 'Museologia', 'Música', 'Nutrição', 'Pedagogia', 'Pedagogia (EaD)', 'Química', 'Química Industrial', 'Serviço Social', 'Sistemas de Informação', 'Turismo', 'Artes Cênicas (Bacharelado)', 'Artes Cênicas (Licenciatura)', 'Ciências Biológicas (Bacharelado)', 'Ciências Biológicas (Licenciatura)', 'Educação Física (Bacharelado)', 'Educação Física (Licenciatura)', 'Filosofia (Bacharelado)', 'Filosofia (Licenciatura)', 'Física (Bacharelado)', 'Física (Licenciatura)', 'História (Bacharelado)', 'História (Licenciatura)', 'Letras (Bacharelado)', 'Letras (Licenciatura)', 'Matemática (Bacharelado)', 'Matemática (Licenciatura)']}
course_df = pd.DataFrame(courses_dict).sort_values(by="code", ascending=True)
course_df["created_at"] = datetime.now()

existing_course_codes = []
# Check if 'course' key exists in existing_data
if "course" in existing_data:
    existing_course_codes = set(existing_data["course"]["code"])

course_df = course_df[~course_df["code"].isin(existing_course_codes)]

store_df(course_df, "course")

In [ ]:
from datetime import datetime

URL = "https://zeppelin10.ufop.br/HorarioAulas/"

desired_departments = ["DECSI", "DECEA", "DEELT", "DEENP", "DEETE"]
selected_semester = "2024/1"


def get_HTML_content_for_semester(url, department=None):
    chrome_options = Options()
    chrome_options.add_argument("--headless")
    driver = webdriver.Chrome(options=chrome_options)
    driver.get(url)

    try:
        # Wait for the semester select element to be present
        semester_select = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "formPrincipal:anoSemestre"))
        )

        # Select the desired semester
        select = Select(semester_select)
        select.select_by_value(selected_semester)

        # Find and click the "Filtrar" button
        filtrar_button = driver.find_element(By.ID, "formPrincipal:botaoEnviar")
        filtrar_button.click()

        # Wait for the table to be updated
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.ID, "formPrincipal:tabela"))
        )

        if department:
            # If a department is specified, click on it
            elem = WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, f"//*[text()='{department}']"))
            )
            elem.click()

            # Wait for the page to load after clicking the department
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.ID, "formPrincipal:tabela"))
            )

        # Get the updated page source
        html_content = driver.page_source
        
    finally:
        driver.quit()

    return BeautifulSoup(html_content, "lxml")


def parse_schedule_string(schedule_string):
    entries = []

    if schedule_string == "":
        return entries

    schedule_parts = schedule_string.split(" / ")

    if len(schedule_parts) == 0:
        schedule_parts.append(schedule_string)

    for part in schedule_parts:
        day, time_info = part.split(" ")
        start_time, end_time = time_info.split("-")
        class_type = end_time[-2]  # T for theoretical, P for practical
        end_time = end_time[:-3]  # Remove the class type from end_time

        entry = {
            "day_of_week": day,
            "start_time": datetime.strptime(start_time, "%H:%M").time(),
            "end_time": datetime.strptime(end_time, "%H:%M").time(),
            "class_type": class_type,
        }
        entries.append(entry)

    return entries


def get_field_list(html_content, field):
    field_list = []
    table = html_content.find("table", {"id": "formPrincipal:tabela"})
    if table:
        tbody = table.find("tbody")
        tr_elements = tbody.find_all("tr")

        for i, tr in enumerate(tr_elements):
            if field == "descricao":
                span = tr.find(
                    "span", {"id": "formPrincipal:tabela:{}:{}".format(i, "disciplina")}
                )
                title = span.find_parent("a").get(
                    "title"
                )  # Extract the 'title' attribute of the parent <a> tag
                field_list.append(title)
                continue

            span = tr.find(
                "span", {"id": "formPrincipal:tabela:{}:{}".format(i, field)}
            )
            field_list.append(span.text)

    return field_list


def get_departments(existing_data):
    soup = get_HTML_content_for_semester(URL)
    
    departments_list = []
    table = soup.find("table", {"id": "formPrincipal:tabela"})
    
    # Create a dictionary of existing departments with code as key and id as value
    existing_departments = existing_data.get("department", pd.DataFrame())
    existing_dept_dict = dict(zip(existing_departments.get("code", []), existing_departments.get("id", [])))
    
    if table:
        tbody = table.find("tbody")
        tr_elements = tbody.find_all("tr")

        for i, tr in enumerate(tr_elements):
            tableCode = tr.find(
                "span", {"id": f"formPrincipal:tabela:{i}:codigoDepartamento"}
            )
            tableName = tr.find(
                "span", {"id": f"formPrincipal:tabela:{i}:descricao"}
            )
            
            code = tableCode.text.strip()
            name = tableName.text.strip()
            
            # Use existing ID if department already exists, otherwise generate new UUID
            dept_id = existing_dept_dict.get(code, str(uuid.uuid4()))

            departments_list.append(
                {
                    "id": dept_id,
                    "code": code,
                    "name": name,
                }
            )

        departments_df = pd.DataFrame(departments_list)
        departments_df["created_at"] = datetime.now()
        return departments_df, departments_list

    return pd.DataFrame(), []

def get_discipline_tables(departments_list, existing_data):
    discipline_dict = {
        "id": [],
        "code": [],
        "name": [],
        "description": [],
        "department_id": [],
        "equivalency_group_id": [],
    }
    class_dict = {
        "id": [],
        "class_number": [],
        "discipline_id": [],
        "professor": [],
        "semester": [],
    }
    schedule_dict = {
        "id": [],
        "discipline_class_id": [],
        "day_of_week": [],
        "start_time": [],
        "end_time": [],
        "class_type": [],
    }

    existing_disciplines = existing_data.get("discipline", pd.DataFrame())
    existing_discipline_codes = set(existing_disciplines["code"]) if not existing_disciplines.empty else set()
    existing_discipline_id_map = dict(zip(existing_disciplines["code"], existing_disciplines["id"])) if not existing_disciplines.empty else {}

    existing_classes = existing_data.get("discipline_class", pd.DataFrame())
    existing_schedules = existing_data.get("discipline_class_schedule", pd.DataFrame())

    discipline_code_to_id = existing_discipline_id_map.copy()
    discipline_name_to_info = defaultdict(list)

    print("\nBuscando disciplinas")
    with tqdm(total=len(departments_list), desc="Progresso", ascii=True) as pbar:
        for department in departments_list:
            if department['code'] not in desired_departments:
                continue

            html_content = get_HTML_content_for_semester(URL, department['code'])
            columns_list = ["codigo", "disciplina", "descricao", "turma", "horario", "professores"]
            columns_dict_list = {column_name: get_field_list(html_content, column_name) for column_name in columns_list}

            for i in range(len(columns_dict_list["codigo"])):
                code = columns_dict_list["codigo"][i]
                name = columns_dict_list["disciplina"][i]
                description = columns_dict_list["descricao"][i]
                class_number = columns_dict_list["turma"][i]
                professor = columns_dict_list["professores"][i]

                if code not in discipline_code_to_id:
                    if code not in existing_discipline_codes:
                        discipline_id = str(uuid.uuid4())
                        discipline_dict["id"].append(discipline_id)
                        discipline_dict["code"].append(code)
                        discipline_dict["name"].append(name)
                        discipline_dict["description"].append(description)
                        discipline_dict["department_id"].append(department['id'])
                        discipline_dict["equivalency_group_id"].append(None)  # We'll update this later
                    else:
                        discipline_id = existing_discipline_id_map[code]
                    
                    discipline_code_to_id[code] = discipline_id

                    discipline_name_to_info[name].append({
                        "id": discipline_id,
                        "code": code,
                        "department_id": department['id']
                    })
                else:
                    discipline_id = discipline_code_to_id[code]

                # Check if class already exists for this discipline and semester
                existing_class = existing_classes[
                    (existing_classes["discipline_id"] == discipline_id) &
                    (existing_classes["class_number"] == class_number) &
                    (existing_classes["semester"] == selected_semester)
                ]

                if existing_class.empty:
                    discipline_class_id = str(uuid.uuid4())
                    class_dict["id"].append(discipline_class_id)
                    class_dict["class_number"].append(class_number)
                    class_dict["discipline_id"].append(discipline_id)
                    class_dict["professor"].append(professor)
                    class_dict["semester"].append(selected_semester)
                else:
                    discipline_class_id = existing_class.iloc[0]["id"]

                schedule_entries = parse_schedule_string(columns_dict_list["horario"][i])
                for entry in schedule_entries:
                    # Check if schedule already exists for this class
                    existing_schedule = existing_schedules[
                        (existing_schedules["discipline_class_id"] == discipline_class_id) &
                        (existing_schedules["day_of_week"] == entry["day_of_week"]) &
                        (existing_schedules["start_time"] == entry["start_time"]) &
                        (existing_schedules["end_time"] == entry["end_time"]) &
                        (existing_schedules["class_type"] == entry["class_type"])
                    ]

                    if existing_schedule.empty:
                        schedule_dict["id"].append(str(uuid.uuid4()))
                        schedule_dict["discipline_class_id"].append(discipline_class_id)
                        schedule_dict["day_of_week"].append(entry["day_of_week"])
                        schedule_dict["start_time"].append(entry["start_time"])
                        schedule_dict["end_time"].append(entry["end_time"])
                        schedule_dict["class_type"].append(entry["class_type"])

            pbar.set_postfix(info=f"Salvando disciplinas de {department['code']}...")
            pbar.update(1)
            
    # Handle equivalencies only for new disciplines
    new_disciplines = [
        {"id": id, "code": code, "name": name, "department_id": dept_id}
        for id, code, name, dept_id in zip(
            discipline_dict["id"], discipline_dict["code"], 
            discipline_dict["name"], discipline_dict["department_id"]
        )
    ]
    equivalency_groups = handle_equivalencies(new_disciplines)

    existing_equivalency_groups = existing_data.get("equivalency_group", pd.DataFrame())
    existing_group_ids = set(existing_equivalency_groups["id"]) if not existing_equivalency_groups.empty else set()

    equivalency_group_dict = {"id": [], "created_at": []}
    for group in equivalency_groups:
        group_id = str(uuid.uuid4())
        if group_id not in existing_group_ids:
            equivalency_group_dict["id"].append(group_id)
            equivalency_group_dict["created_at"].append(datetime.now())

        for discipline in group:
            if discipline["id"] in discipline_dict["id"]:
                index = discipline_dict["id"].index(discipline["id"])
                discipline_dict["equivalency_group_id"][index] = group_id

    discipline_df = pd.DataFrame(discipline_dict)
    equivalency_group_df = pd.DataFrame(equivalency_group_dict)
    class_df = pd.DataFrame(class_dict)
    schedule_df = pd.DataFrame(schedule_dict)

    currentTime = datetime.now()

    discipline_df["created_at"] = currentTime
    class_df["created_at"] = currentTime
    schedule_df["created_at"] = currentTime

    return discipline_df, class_df, schedule_df, equivalency_group_df

# Process departments
department_df, departments_list = get_departments(existing_data)
existing_department_codes = set(existing_data.get("department", {}).get("code", []))
print(f"Existing department codes: {existing_department_codes}")

new_department_df = department_df[~department_df["code"].isin(existing_department_codes)]
print(f"New departments to be added: {new_department_df}")

# Store new departments
store_df(new_department_df, "department")
print("New departments saved successfully!")

# Update the departments_list with the correct IDs (including existing ones)
departments_list = department_df.to_dict('records')

discipline_df, discipline_class_df, discipline_class_schedule_df, equivalency_group_df = get_discipline_tables(departments_list, existing_data)

# Store new equivalency groups
store_df(equivalency_group_df, 'equivalency_group')
print("Novos grupos de equivalência salvos com sucesso!")

# Store new disciplines
new_discipline_df = discipline_df[~discipline_df["code"].isin(existing_data.get("discipline", {}).get("code", []))]
remaining_discipline_ids = set(new_discipline_df["id"])
store_df(new_discipline_df, "discipline")
print("Novas disciplinas salvas com sucesso!")

# Store new discipline classes for the current semester
existing_classes = existing_data.get("discipline_class", pd.DataFrame())
new_discipline_class_df = discipline_class_df[~discipline_class_df["id"].isin(existing_classes["id"])]
store_df(new_discipline_class_df, "discipline_class")

# Store new discipline class schedules
existing_schedules = existing_data.get("discipline_class_schedule", pd.DataFrame())
new_discipline_class_schedule_df = discipline_class_schedule_df[~discipline_class_schedule_df["id"].isin(existing_schedules["id"])]
store_df(new_discipline_class_schedule_df, "discipline_class_schedule")

print("Classes e horários das disciplinas salvos com sucesso!")

In [ ]:
import tabula
import pandas as pd
import uuid
import re

code_pattern = r"[A-Z]{3}\d{3}"
subject_pattern = r"\b[A-Z]+\b"
classes_pattern = r"^(T P|T|P)$"
prerequisite_pattern = r"[A-Z]{3}\d{3}|\d+\s+horas"
chs_che_pattern = r"^\d+\/\d+$"

discipline_course_dict = {"id": [], "discipline_id": [], "course_id": [], "period": [], "mandatory": [], "created_at": []}
prerequisite_dict = {"id": [], "discipline_course_id": [], "prerequisite_discipline_id": [], "created_at": []}
empty_discipline_course_df = pd.DataFrame(data=discipline_course_dict)
empty_prerequisite_df = pd.DataFrame(data=prerequisite_dict)

def get_col_idx(df_value, pattern):
    indexes = []
    for i, item in enumerate(df_value):
        if i == 0:
            continue
        match = re.search(pattern, str(item), re.UNICODE)
        if match is not None:
            indexes.append(i)

    return indexes


def get_prerequisites(df_value):
    prerequisites = []
    prereq_idx = get_col_idx(df_value, prerequisite_pattern)
    if len(prereq_idx) > 0:
        prerequisites = [df_value[i] for i in prereq_idx]

    return format_prerequisites(prerequisites)


def get_discipline(df_value):
    subject_idx = get_col_idx(df_value, subject_pattern)
    if len(subject_idx) > 0:
        return re.sub(code_pattern, "", df_value[subject_idx[0]])
    return ""


def get_chs_che(df_value):
    chs_che_idx = get_col_idx(df_value, chs_che_pattern)

    if len(chs_che_idx) > 0:
        chs_che_list = df_value[chs_che_idx[0]].split("/")
        return tuple(map(int, chs_che_list))

    return ("", "")


def get_classes(df_value, classes_idx):
    classes = []
    if len(classes_idx) > 0:
        classes = [df_value[i] for i in classes_idx]
        return " ".join(classes)

    return ""

def get_period(df_value, mandatory):
    if not mandatory or not df_value[0]:
        return ""

    for item in reversed(df_value):
        if item:
            return int(item)
    return ""


def get_cha(df_value, chs):
    if not chs:
        return ""

    chs_alt = chs * 1.2

    if len(df_value) == 0:
        return ""

    for text in df_value:
        try:
            formatted_text = int(text)
            if formatted_text == chs or formatted_text == chs_alt:
                return formatted_text
        except ValueError:
            continue

    return ""


def format_prerequisites(df_value):
    if len(df_value) == 0:
        return ""

    requisites = []
    for text in df_value:
        if not text:
            continue

        matches = re.findall(code_pattern, text)
        requisites.extend(matches)

    joined_matches = " ".join(requisites)
    return joined_matches


def get_prerequisite_df(discipline_course_df):
    prerequisite_dict = {"id": [], "discipline_course_id": [], "prerequisite_discipline_id": []}
    discipline_course_dict = discipline_course_df.to_dict('records')

    for discipline_course in discipline_course_dict:   
        prerequisites = discipline_course['prerequisites'].split()
         # create a dictionary where the keys are the codes and the values are the ids
        id_map = discipline_df.set_index('code')['id'].to_dict()

        # use the dictionary to map the codes to ids
        discipline_ids = [id_map[code] for code in prerequisites if code in id_map]
        if len(discipline_ids) > 0:
            for prerequisite in discipline_ids:
                prerequisite_id = uuid.uuid4()
                prerequisite_dict['id'].append(prerequisite_id)
                prerequisite_dict['discipline_course_id'].append(discipline_course['id'])
                prerequisite_dict['prerequisite_discipline_id'].append(prerequisite)
   

    prerequisite_df = pd.DataFrame(prerequisite_dict)
    return prerequisite_df


def get_discipline_course_tables(df, course_id, mandatory):
    df = df.replace({r"\r": " "}, regex=True)
    df_struct = {"id": [], "discipline_id": [], "period": [], "prerequisites": []}
    discipline_course_df = pd.DataFrame(data=df_struct)

    iterIdx = -1

    for idx, value in enumerate(df.values):
        if idx == 0:
            iterIdx = -1
            discipline_course_df.at[0, "id"] = ""
            discipline_course_df.at[0, "discipline_id"] = ""
            discipline_course_df.at[0, "period"] = ""
            discipline_course_df.at[0, "prerequisites"] = ""
            continue

        discipline_ids = discipline_df[discipline_df['code'] == value[0]]['id']
        discipline_course_id = ''
        discipline_id = ''

        if(len(discipline_ids.values) > 0):
            discipline_id = discipline_ids.values[0]
        
        if value[0]:
            discipline_course_id = str(uuid.uuid4())

        discipline_course_df.at[idx, "id"] = discipline_course_id
        discipline_course_df.at[idx, "discipline_id"] = discipline_id
        discipline_course_df.at[idx, "period"] = get_period(value, mandatory)
        discipline_course_df.at[idx, "prerequisites"] = get_prerequisites(value)

        if value[0] != "":
            iterIdx = -1
            continue

        if iterIdx == -1:
            iterIdx = idx - 1

        discipline_course_df.loc[iterIdx, :] = [
            f"{item1} {item2}".strip()
            for item1, item2 in zip(
                discipline_course_df.values[iterIdx], discipline_course_df.values[idx]
            )
        ]

    discipline_course_df.drop(
        discipline_course_df[discipline_course_df["discipline_id"] == ""].index,
        inplace=True,
    )
    discipline_course_df["mandatory"] = mandatory
    discipline_course_df["course_id"] = course_id
    discipline_course_df['period'] = discipline_course_df['period'].replace("", 0)

    prerequisite_df = pd.DataFrame()
    if not discipline_course_df.empty:
        prerequisite_df = get_prerequisite_df(discipline_course_df)

    discipline_course_df.drop("prerequisites", axis=1, inplace=True)
    return discipline_course_df, prerequisite_df


def scrape_table_from_pdf(pdf_path):
    
    # Read the table from the PDF file
    df_list = tabula.read_pdf(pdf_path, pages="all")

    course_code = os.path.splitext(os.path.basename(pdf_path))[0]
    course_ids = course_df[course_df['code'] == course_code]['id']

    
    if(len(course_ids.values) == 0):
        
        return empty_discipline_course_df, empty_prerequisite_df

    course_id = course_ids.values[0]
    discipline_course_dfs = []
    prerequisite_dfs = []

    for df in df_list:
        df.fillna("", inplace=True)
        header = df.columns.to_list()
       
        # Filter and select the desired columns based on the header
        if "DISCIPLINAS OBRIGATÓRIAS" in header:
            discipline_course_df, prerequisite_df = get_discipline_course_tables(
                df, course_id, True
            )
            discipline_course_dfs.append(discipline_course_df)
            prerequisite_dfs.append(prerequisite_df)

        elif (
            "DISCIPLINAS ELETIVAS" in header
            or "DISCIPLINAS ELETIVAS PRÉ-REQUISITO" in header
        ):
            discipline_course_df, prerequisite_df = get_discipline_course_tables(
                df, course_id, False
            )
            discipline_course_dfs.append(discipline_course_df)
            prerequisite_dfs.append(prerequisite_df)
        else:
            continue

    combined_discipline_course_df = pd.concat(discipline_course_dfs, ignore_index=True)
    combined_prerequisite_df = pd.concat(prerequisite_dfs, ignore_index=True)

    discipline_course_df = combined_discipline_course_df[
        combined_discipline_course_df["discipline_id"].isin(discipline_df["id"])
    ]

    prerequisite_df = combined_prerequisite_df
    if not combined_prerequisite_df.empty:
        prerequisite_df = combined_prerequisite_df[
            combined_prerequisite_df["discipline_course_id"].isin(
                discipline_course_df["id"]
            )
        ]
    discipline_course_df["created_at"] = datetime.now()
    prerequisite_df["created_at"] = datetime.now()

    return discipline_course_df, prerequisite_df


files = os.listdir(pdfs_folder_path)
pdf_files = [
    os.path.join(pdfs_folder_path, file) for file in files if file.endswith(".pdf")
]

for pdf_file in [
    "./courses_pdfs/engenharia-de-computacao.pdf",
    "./courses_pdfs/engenharia-de-producao-jm.pdf",
    "./courses_pdfs/sistemas-de-informacao.pdf",
    "./courses_pdfs/engenharia-eletrica.pdf",
]:
    print(f"Buscando disciplinas de {pdf_file}")
    discipline_course_df, prerequisite_df = scrape_table_from_pdf(pdf_file)
    
    discipline_course_df = discipline_course_df[discipline_course_df['discipline_id'].isin(remaining_discipline_ids)]
    store_df(discipline_course_df, 'discipline_course')
    print("Disciplinas do curso armazenadas com sucesso!")

    remaining_discipline_course_ids = set(discipline_course_df['id'])
    prerequisite_df = prerequisite_df[prerequisite_df['discipline_course_id'].isin(remaining_discipline_course_ids)]
    store_df(prerequisite_df, 'prerequisite')
    print("Prerequisitos armazenados com sucesso!\n")

